In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
from IPython.display import HTML

sagemaker.__version__

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'2.239.0'

In [ ]:
%store -r 

%store

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-12-06-30-53'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-igloxuzrs3z2'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-11-21-33-07'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow

In [3]:
sm = boto3.client("sagemaker")

In [ ]:
print(project_name)
print(project_id)

mlops-02-14-11-03-33
p-ca7phmcraepa


## Explore the project in the Studio UI

In [5]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

In [20]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [21]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [24]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [25]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

---

## Working with MLOps project for model deployment

### Approve a model version

In [32]:
# list all model packages and select the latest one
model_packages = [
    p["ModelPackageSummaryList"] for p in
    sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=f'{project_name}-{project_id}',
        ModelApprovalStatus='PendingManualApproval',
        SortBy="CreationTime",
        SortOrder="Descending",
    )
][0]

In [33]:
if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

There are 2 model versions in the from-idea-to-prod-pipeline-model-12-06-30-22 model package group
Approve the most recent model package:
arn:aws:sagemaker:us-east-1:906545278380:model-package/mlops-02-14-11-03-33-p-ca7phmcraepa/2


In [34]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

In [40]:
# Show the model package version link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions/version-{}/overview">the model package version</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id, latest_model_package_arn.split('/')[-1]))
)

### Deployment pipeline execution

In [41]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

### Deploy the model version to production

In [49]:
while sm.describe_endpoint(EndpointName=f'{project_name}-staging')['EndpointStatus'] != 'InService':
    sleep(10)
    print(f'Waiting untile the endpoint {project_name}-staging is operational')

print(f'Endpoint {project_name}-staging is InService')

Endpoint mlops-02-14-11-03-33-staging is InService


In [57]:
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-{}-modeldeploy/view?region={}">AWS CodePipeline</a></b>'.format(
            project_name, project_id, region)
    )
)

In [58]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

In [59]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/endpoints">endpoints of the project {}</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_name))
)